In [10]:
import ipywidgets as widgets
from IPython.display import display
from stmpy import Machine, Driver
import paho.mqtt.client as mqtt
from threading import Thread
import logging

class Tick:
    def __init__(self):
        self.ticks = 0
        self.tocks = 0

    def send_mqtt_tick(self):
        print("Tick! {}".format(self.ticks))
        self.ticks = self.ticks + 1
        self.mqtt_client.publish("tick", self.ticks)

    def send_mqtt_tock(self):
        print("Tock! {}".format(self.tocks))
        self.tocks = self.tocks + 1
        self.mqtt_client.publish("tock", self.ticks)

class ChargingStation:
    
    def load_images(self):
        self.free = open("images/green_on.png", "rb").read()
        self.charging = open("images/yellow_on.png", "rb").read()
        self.full = open("images/red_on.png", "rb").read()
        
    def on_button_start(self, b):
        self.stm.send('start')
        
    def on_button_stop(self, b):
        self.stm.send('stop')
        
    def display(self):
        self.button_start = widgets.Button(description="Start")
        self.button_start.on_click(self.on_button_start)
        self.button_stop = widgets.Button(description="Stop")
        self.button_stop.on_click(self.on_button_stop)
        self.station = widgets.Image(value=self.free, format='png', width=50, height=50)
        display(self.station, self.button_start, self.button_stop)
        
    def __init__(self):
        self.load_images()
        self.display()
        self.mqtt_client = mqtt.Client(callback_api_version = mqtt.CallbackAPIVersion.VERSION1)
        self.mqtt_client.on_connect = self.on_mqtt_connect
        self.mqtt_client.on_message = self.on_mqtt_message
        self.mqtt_client.connect("mqtt://broker.hivemq.com", 1883)
        self.mqtt_client.subscribe("charging_status")
        self.mqtt_client.subscribe("reserved")
        
    def turn_free_on(self):
        self.station.set_trait(name='value', value=self.free)
        
    def turn_charging_on(self):
        self.station.set_trait(name='value', value=self.charging)
        
    def turn_full_on(self):
        self.station.set_trait(name='value', value=self.full)
        
    def on_mqtt_connect(self, client, userdata, flags, rc):
        logging.info("Connected to MQTT broker with result code "+str(rc))
        
    def on_mqtt_message(self, client, userdata, message):
        topic = message.topic
        payload = message.payload.decode()
        if topic == "charging_status" and payload == "start":
            self.stm.send("start")
        elif topic == "reserved" and payload == "reserved":
            self.stm.send("reserved")


traffic = ChargingStation()
# Definerer tilstandene for ladestasjonen
t0 = {'source': 'initial', 'target': 'free'}
t1 = {'trigger':'start', 'source':'free', 'target':'charging', 'effect':'turn_charging_on'}
t2 = {'trigger':'stop', 'source':'charging', 'target':'full', 'effect':'turn_full_on'}
t3 = {'trigger':'start', 'source':'full', 'target':'free', 'effect':'turn_free_on'}
t4 = {'trigger':'reserved', 'source':'free', 'target':'full', 'effect':'turn_full_on'}

stm_traffic = Machine(name='stm_traffic', transitions=[t0, t1, t2, t3, t4], obj=traffic)

traffic.stm = stm_traffic

driver = Driver()
driver.add_machine(stm_traffic)
driver.start()

logger = logging.getLogger('stmpy.Driver')
logger.setLevel(logging.INFO)
ch = logging.StreamHandler()
ch.setLevel(logging.INFO)
formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
ch.setFormatter(formatter)
logger.addHandler(ch)

logger = logging.getLogger('stmpy.Machine')
logger.setLevel(logging.INFO)
ch = logging.StreamHandler()
ch.setLevel(logging.INFO)
formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
ch.setFormatter(formatter)
logger.addHandler(ch)

%config Application.log_level="INFO"

Image(value=b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\x00O\x00\x00\x00O\x08\x02\x00\x00\x00%V\x0f\xd1\x00\…

Button(description='Start', style=ButtonStyle())

Button(description='Stop', style=ButtonStyle())

/var/folders/7h/sbcvcpps61951x9b2lxcqd8w0000gn/T/ipykernel_81273/1960982729.py:47: DeprecationWarning: Callback API version 1 is deprecated, update to latest version
  self.mqtt_client = mqtt.Client(callback_api_version = mqtt.CallbackAPIVersion.VERSION1)


gaierror: [Errno 8] nodename nor servname provided, or not known